In [1]:
import os
import numpy as np
import pandas as pd
import dask.dataframe as dd
from knowledge_graph_generator import KnowledgeGraphGenerator
from fusion import TransE, TransEFuser, MovieKG

import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader

from sklearn.preprocessing import MinMaxScaler

In [2]:
full_df = pd.read_csv("data/kg_count_year_df.csv")
df = full_df.loc[full_df["arc_count"] >= 500].reset_index(
    drop=True
)
df = df[["entity_id", "relation", "value"]]
kg_obj = KnowledgeGraphGenerator(known_data_list=[df])
fuser = TransEFuser(kg_obj)
kg_df = fuser.fuse()

relation is, epoch 0, loss 1.051262378692627


KeyboardInterrupt: 

In [ ]:
transe = TransE(kg_obj)

count_limit_250_df = pd.read_csv("data/count_limit_250.csv")

In [3]:
count_limit_100_df = pd.read_csv("data/count_limit_100.csv")

In [7]:
kg_obj = KnowledgeGraphGenerator(known_data_list = [count_limit_100_df[["entity_id", "relation", "value"]]])
transe = TransE(kg_obj)

In [39]:
c[count_limit_100_df.columns].to_csv("data/count_limit_100.csv", index=False)

In [36]:
c = count_limit_100_df.copy(deep=True)
g = count_limit_100_df[["relation", "value", "distance"]].groupby(["relation", "value"])

In [37]:
c = c.merge(g.min().reset_index().rename(columns={"distance": "min_distance"}), on=["relation", "value"])
c = c.merge(g.max().reset_index().rename(columns={"distance": "max_distance"}), on=["relation", "value"])
c["probability"] = 1 - (c["distance"] - c["min_distance"])/(c["max_distance"] - c["min_distance"])

In [38]:
c[(c["relation"] == "is")&(c["value"] == "good")].sort_values("probability")

,entity_id,relation,value,distance,probability,min_distance,max_distance
5238,m/gay_divorcee,is,good,5.490583,0.000000,3.561836,5.490583
6856,m/lootera,is,good,5.481771,0.004568,3.561836,5.490583
6772,m/mating_habits_of_the_earthbound_human,is,good,5.467676,0.011877,3.561836,5.490583
5304,m/into_the_night,is,good,5.465469,0.013021,3.561836,5.490583
6308,m/passenger_57,is,good,5.465423,0.013045,3.561836,5.490583
...,...,...,...,...,...,...,...
2117,m/ghost_in_the_shell_2017,is,good,3.568491,0.996549,3.561836,5.490583
2873,m/mad_max_fury_road,is,good,3.567918,0.996847,3.561836,5.490583
1825,m/dunkirk_2017,is,good,3.565719,0.997987,3.561836,5.490583
2507,m/inside_out_2015,is,good,3.565609,0.998044,3.561836,5.490583


In [10]:
for relation in count_limit_250_df["relation"].unique():
    for value in tqdm(count_limit_250_df[count_limit_250_df["relation"] == relation]["value"].unique()):
        d = count_limit_250_df[(count_limit_250_df["relation"] == relation)&(count_limit_250_df["value"] == value)]["distance"]
        scaler = MinMaxScaler()
        p = scaler.fit_transform(d.values.reshape(-1, 1))
        count_limit_250_df.loc[(count_limit_250_df["relation"] == relation)&(count_limit_250_df["value"] == value), "probability"]

KeyboardInterrupt: 

In [3]:
r = df.loc[df["relation"] == kg_obj.relations[0]]
idx_array = transe.gen_training_array(r)
transe.gen_random_dist(idx_array)
d = count_limit_250_df[count_limit_250_df["relation"] == kg_obj.relations[0]]["distance"]
p = transe.gen_probability(d)

In [3]:
df = full_df.loc[full_df["arc_count"] >= 250].reset_index(
    drop=True
)

In [9]:
count_limit_250_df = pd.read_csv("data/count_limit_250.csv")

In [4]:
d = df.loc[(df["relation"] == "is")]["distance"].values
p = df.loc[(df["relation"] == "is")]["probability"].values

In [5]:
np.max(p)

0.9786944933259916

In [25]:
sns.scatterplot(d, p)

C:\Users\TomPace\Anaconda3\lib\site-packages\seaborn\_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


<AxesSubplot:>

In [2]:
df = pd.read_csv("data/kg_count_year_df.csv")
# df = df.loc[(df["year_released"] == 2017)&(df["arc_count"] >= 500)].reset_index(drop=True)
# toy = df.loc[(df["year_released"] == 2017)&(df["arc_count"] >= 25)]
toy = df.loc[(df["arc_count"] >= 250)]
toy = toy[["entity_id", "relation", "value"]]

# kg_obj = KnowledgeGraphGenerator(known_data_list=[toy])
# fuser = TransEFuser(kg_obj, max_epochs=1, batch_size=1024)